# 01 PCA Macro Factors

Extract factors and interpret loadings.


## Table of Contents
- [Standardize](#standardize)
- [Fit PCA](#fit-pca)
- [Interpret loadings](#interpret-loadings)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Unsupervised notebooks help you understand macro structure:
- latent factors (PCA),
- regimes (clustering),
- anomalies/crises.


## Prerequisites (Quick Self-Check)
- Completed Part 01 (macro panel) or equivalent.
- Comfort with standardization and basic linear algebra intuition (variance, distance).

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Forgetting to standardize features before PCA/clustering.
- Over-interpreting clusters as causal regimes.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/04_unsupervised/01_pca_macro_factors.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/04_unsupervised/01_pca_macro_factors.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Use PCA to extract a small number of macro factors from many indicators.

Why PCA is useful here:
- macro series are correlated
- PCA creates orthogonal (uncorrelated) components
- components can act like "macro factors" (growth, inflation, rates, etc.)



## Primer: pandas time series essentials (indexing, resampling, lags)

Most “mysterious bugs” in time series work come from index and alignment mistakes. This primer gives you the minimum patterns to avoid them.

### 1) DatetimeIndex (the first thing to verify)

Most time-series operations assume a `DatetimeIndex`:

```python
import pandas as pd

df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()
assert isinstance(df.index, pd.DatetimeIndex)
```

**Expected output / sanity checks**
- `df.index.min(), df.index.max()` look reasonable
- `df.index.is_monotonic_increasing` is `True`

### 2) Resampling (frequency alignment)

Resampling converts one frequency to another. Choose the aggregation rule intentionally.

```python
# month-end last value (end-of-period)
df_me_last = df.resample("ME").last()

# month-end mean (average-of-period)
df_me_mean = df.resample("ME").mean()

# quarter-end mean
df_q_mean = df.resample("QE").mean()
```

**Interpretation matters**
- `.last()` treats end-of-period value as “the period’s value.”
- `.mean()` treats the period average as “the period’s value.”

### 3) Alignment and merging

When joining series, always check missingness after the join:

```python
merged = df1.join(df2, how="outer").sort_index()
print(merged.isna().sum().sort_values(ascending=False).head(10))
```

### 4) Lags and rolling windows (watch for leakage!)

```python
# lag 1 period (past-only)
df["x_lag1"] = df["x"].shift(1)

# rolling mean using past values ending at t
df["x_roll12"] = df["x"].rolling(12).mean()
```

**Leakage pitfalls**
- `shift(-1)` uses the future.
- `rolling(..., center=True)` uses the future.

### 5) A quick workflow you should repeat

1) Set and verify DatetimeIndex.
2) Resample intentionally (mean vs last).
3) Join and inspect missingness.
4) Add lags/rolls (past-only).
5) `dropna()` to build a clean modeling table.


<a id="standardize"></a>
## Standardize

### Goal
Load a monthly panel and standardize features (mean 0, std 1).

PCA is sensitive to scale: if one variable has larger units, it can dominate the components.



### Your Turn (1): Load panel_monthly.csv (or sample)


In [ ]:
import pandas as pd

path = PROCESSED_DIR / 'panel_monthly.csv'
if path.exists():
    df = pd.read_csv(path, index_col=0, parse_dates=True)
else:
    df = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

df.head()



### Your Turn (2): Build X and standardize


In [ ]:
from sklearn.preprocessing import StandardScaler

x_cols = df.columns.tolist()
X = df[x_cols].dropna().copy()

sc = StandardScaler().fit(X)
X_s = sc.transform(X)

# TODO: Validate standardization
...



<a id="fit-pca"></a>
## Fit PCA

### Goal
Fit PCA and inspect explained variance.



### Your Turn (1): Fit PCA


In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

pca = PCA(n_components=3).fit(X_s)

evr = pd.Series(pca.explained_variance_ratio_, index=[f'PC{i+1}' for i in range(pca.n_components_)])
evr



### Your Turn (2): Scree plot


In [ ]:
import matplotlib.pyplot as plt

# TODO: Plot explained variance ratio by component.
...



<a id="interpret-loadings"></a>
## Interpret loadings

### Goal
Interpret what each component represents in economic terms.

Loadings tell you which original variables contribute most to each component.



### Your Turn (1): Build a loadings table


In [ ]:
import pandas as pd

loadings = pd.DataFrame(
    pca.components_.T,
    index=x_cols,
    columns=[f'PC{i+1}' for i in range(pca.n_components_)],
)

# TODO: For each PC, list the top + and top - loadings.
...



### Your Turn (2): Name the components


In [ ]:
# TODO: Write a name/interpretation for PC1 and PC2.
notes = """
PC1: ...
PC2: ...
"""
print(notes)



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
# TODO: Confirm your feature matrix is standardized (or justify why not).
# Example:
# assert abs(X_scaled.mean(axis=0)).max() < 1e-6
# assert abs(X_scaled.std(axis=0) - 1).max() < 1e-6
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Standardize</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_pca_macro_factors — Standardize
import pandas as pd
from sklearn.preprocessing import StandardScaler

panel = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True).dropna()
X = StandardScaler().fit_transform(panel)
```

</details>

<details><summary>Solution: Fit PCA</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_pca_macro_factors — Fit PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3).fit(X)
pca.explained_variance_ratio_
```

</details>

<details><summary>Solution: Interpret loadings</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_pca_macro_factors — Interpret loadings
import pandas as pd
loadings = pd.DataFrame(pca.components_.T, index=panel.columns, columns=[f'PC{i+1}' for i in range(pca.n_components_)])
loadings.sort_values('PC1', key=abs, ascending=False).head(10)
```

</details>

